# Machine Problem  
Cruel, Nathaniel and Virtucio, Renzo  
Department of Computer Science, University of the Philippines Diliman  
CS 180: Artificial Intelligence  
Professor Carlo Raquel  
June 15, 2021
***

## Introduction to the Problem
## Rationale
## Objective

## Methodology

## Data and Analysis

## Conclusion

## Individual Contributions